In [ ]:
"""
GTF.py
Kamil Slowikowski
December 24, 2013
Read GFF/GTF files. Works with gzip compressed files and pandas.
    http://useast.ensembl.org/info/website/upload/gff.html
LICENSE
This is free and unencumbered software released into the public domain.
Anyone is free to copy, modify, publish, use, compile, sell, or
distribute this software, either in source code form or as a compiled
binary, for any purpose, commercial or non-commercial, and by any
means.
In jurisdictions that recognize copyright laws, the author or authors
of this software dedicate any and all copyright interest in the
software to the public domain. We make this dedication for the benefit
of the public at large and to the detriment of our heirs and
successors. We intend this dedication to be an overt act of
relinquishment in perpetuity of all present and future rights to this
software under copyright law.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.
For more information, please refer to <http://unlicense.org/>
"""
from collections import defaultdict
import gzip
import pandas as pd
import re
import csv


GTF_HEADER  = ['seqname', 'source', 'feature', 'start', 'end', 'score',
               'strand', 'frame']
R_SEMICOLON = re.compile(r'\s*;\s*')
R_COMMA     = re.compile(r'\s*,\s*')
R_KEYVALUE  = re.compile(r'(\s+|\s*=\s*)')


def dataframe(filename):
    """Open an optionally gzipped GTF file and return a pandas.DataFrame.
    """
    # Each column is a list stored as a value in this dict.
    result = defaultdict(list)

    for i, line in enumerate(lines(filename)):
        for key in line.keys():
            # This key has not been seen yet, so set it to None for all
            # previous lines.
            if key not in result:
                result[key] = [None] * i

        # Ensure this row has some value for each column.
        for key in result.keys():
            result[key].append(line.get(key, None))

    return pd.DataFrame(result)


def lines(filename):
    """Open an optionally gzipped GTF file and generate a dict for each line.
    """
    fn_open = gzip.open if filename.endswith('.gz') else open

    with fn_open(filename) as fh:
        for line in fh:
            if line.startswith('#'):
                continue
            else:
                yield parse(line)


def parse(line):
    """Parse a single GTF line and return a dict.
    """
    result = {}

    fields = line.rstrip().split('\t')

    for i, col in enumerate(GTF_HEADER):
        result[col] = _get_value(fields[i])

    # INFO field consists of "key1=value;key2=value;...".
    infos = [x for x in re.split(R_SEMICOLON, fields[8]) if x.strip()]

    for i, info in enumerate(infos, 1):
        # It should be key="value".
        try:
            key, _, value = re.split(R_KEYVALUE, info, 1)
        # But sometimes it is just "value".
        except ValueError:
            key = 'INFO{}'.format(i)
            value = info
        # Ignore the field if there is no value.
        if value:
            result[key] = _get_value(value)

    return result


def _get_value(value):
    if not value:
        return None

    # Strip double and single quotes.
    value = value.strip('"\'')

    # Return a list if the value has a comma.
    if ',' in value:
        value = re.split(R_COMMA, value)
    # These values are equivalent to None.
    elif value in ['', '.', 'NA']:
        return None

    return value

In [ ]:
#Convert the GFT file into a GCT file with all the 'Late' conditions
Raw_Data=['66.gtf','29.gtf','58.gtf','18.gtf','35.gtf','21.gtf','89.gtf','87.gtf','95.gtf','97.gtf','76.gtf','77.gtf']
Data=[]
for i in range(len(Raw_Data)): #Convert and add each GTF file to a panda DataFrame
    Data.append(dataframe(Raw_Data[i])) #Use GTF.py describe above to convert the GTF file into a panda DataFrame
    Data[i]=Data[i][(Data[i].feature=='transcript')] #Keep only tge lines referring to a transcript
    Data[i]=Data[i].drop(['seqname', 'source','feature', 'start', 'end', 'score', 'strand', 'frame', 'transcript_id','reference_id', 'cov', 'TPM', 'exon_number', 'gene_id', 'ref_gene_name'], axis=1)
    #Delete all the columns exept the Gene ID and its relative expression
    Data[i]=Data[i].loc[:,['ref_gene_id','FPKM']] #Reorganize the columns 
    Data[i]=Data[i].rename(columns={'ref_gene_id': 'NAME'}) #Rename the columns containing the gene ID 
    Data[i]=Data[i].dropna(subset=['NAME']) #Delete the rows without a gene ID
    Data[i]=Data[i].drop_duplicates(subset='NAME',keep='first') #Onnly keep the first expression value for a gene ID
    Data[i]=Data[i].rename(columns={'FPKM': Raw_Data[i]}) #Change the name of the column containing the expression value 
    #to have the information about the condition 
    Data[i]=Data[i].set_index('NAME') #Set the Gene ID as the index to allow the concatenation of the different DataFrame
    #based on the gene ID
Final=pd.concat(Data, axis=1) #Merge the list of DataFrame to a final DataFrame containg all conditions
Final.to_csv("Late.txt", sep="\t", quoting=csv.QUOTE_NONE) #Save the file in a tab delimited .xt format

In [ ]:
#Convert the GFT file into a GCT file with all the 'Early' conditions
Raw_Data=['536.gtf','484.gtf','519.gtf','522.gtf','608.gtf','614.gtf','512.gtf','505.gtf','461.gtf','623.gtf','583.gtf','581.gtf']
Data=[]
for i in range(len(Raw_Data)):
    Data.append(dataframe(Raw_Data[i]))
    Data[i]=Data[i][(Data[i].feature=='transcript')]
    Data[i]=Data[i].drop(['seqname', 'source','feature', 'start', 'end', 'score', 'strand', 'frame', 'transcript_id','reference_id', 'cov', 'TPM', 'exon_number', 'gene_id', 'ref_gene_name'], axis=1)
    Data[i]=Data[i].loc[:,['ref_gene_id','FPKM']]
    Data[i]=Data[i].rename(columns={'ref_gene_id': 'NAME'})
    Data[i]=Data[i].dropna(subset=['NAME'])
    Data[i]=Data[i].drop_duplicates(subset='NAME',keep='first')
    Data[i]=Data[i].rename(columns={'FPKM': Raw_Data[i]})
    Data[i]=Data[i].set_index('NAME')
Final=pd.concat(Data, axis=1)
Final.to_csv("Early.txt", sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
#Convert the GFT file into a GCT file with all the extracted 'Cells' conditions
Raw_Data=['CtrlA.gtf','CtrlB.gtf','sh1A.gtf','sh1C.gtf','sh2A.gtf','sh2B.gtf']
Data=[]
for i in range(len(Raw_Data)):
    Data.append(dataframe(Raw_Data[i]))
    Data[i]=Data[i][(Data[i].feature=='transcript')]
    Data[i]=Data[i].drop(['seqname', 'source','feature', 'start', 'end', 'score', 'strand', 'frame', 'transcript_id','reference_id', 'cov', 'TPM', 'exon_number', 'gene_id', 'ref_gene_name'], axis=1)
    Data[i]=Data[i].loc[:,['ref_gene_id','FPKM']]
    Data[i]=Data[i].rename(columns={'ref_gene_id': 'NAME'})
    Data[i]=Data[i].dropna(subset=['NAME'])
    Data[i]=Data[i].drop_duplicates(subset='NAME',keep='first')
    Data[i]=Data[i].rename(columns={'FPKM': Raw_Data[i]})
    Data[i]=Data[i].set_index('NAME')
Final=pd.concat(Data, axis=1)
Final.to_csv("Cells.txt", sep="\t", quoting=csv.QUOTE_NONE)